## Step 5: Create RAG Application Classes

In this step, we will create two Python classes to build a Retrieval-Augmented Generation (RAG) pipeline:

1. **`CortexSearchRetriever`**:
   - This class interacts with the Cortex Search Service to retrieve relevant contextual information based on a user query.
   - It connects to the Cortex Search Service using Snowflake's `Root` object and performs a search with the specified query and result limit.
   - The retrieved context (a list of relevant chunks) will be used to generate prompts for LLMs.

2. **`RAGWithObservability`**:
   - This class integrates the retrieval functionality with a specified Large Language Model (LLM) to complete the RAG pipeline.
   - It uses the retriever to fetch context, creates a structured prompt by combining the context with the user query, and generates a response using the Snowflake Cortex `COMPLETE` function.
   - The class allows testing of different LLMs (e.g., `llama3.1-8b`, `mistral-7b`, `claude-3-5-sonnet`) by specifying the desired model.
    - In the below cells you'll notice an **@instrument** decorator above each function in our RAGWithObservability class
    - This tells Trulens which stages of our application we want to track so we can understand how data flows through our application
    - For example if our query takes 10s to run - what portion of that 10s was spent on retrieval? On prompt augmentation? On completion generation?
        - This becomes increasingly important for complex GenAI applications (i.e. multi-agent apps)
    - After we pass prompts through our trulens recorder we will inspect these traces and spans!

### Workflow Summary:
1. The `CortexSearchRetriever` retrieves relevant context from the Cortex Search Service.
2. The `RAGWithObservability` uses this context to create prompts and generate responses with the specified LLM.

These two classes work together to streamline the RAG pipeline, enabling efficient retrieval and response generation for various use cases.

In [ ]:
# Define the retriever class for interacting with the Cortex Search Service

# Complete the missing code (???) to:
## Specify your database 'SKO_SKORAGHOP_LIVE_PROD', your schema 'HOP', and your Cortex Search Service named 'RAG_SEARCH_SERVICE'
## Specify your SEARCH_COL as the column of interest
## Intialize retriever with your CortexSearchRetriever class
## Use "What are some components of the Snowflake Cortex offering? How do they work?" for the test_query

from typing import List
from snowflake.snowpark import Session
from snowflake.core import Root

# CortexSearchRetriever
class CortexSearchRetriever:
    def __init__(self, session: Session, limit_to_retrieve: int = 4):
        self._session = session
        self._limit_to_retrieve = limit_to_retrieve
        

    def retrieve(self, query: str) -> List[str]:
        root = Root(session)
        cortex_search_service = (
            root
            .databases["SKO_SKORAGHOP_LIVE_PROD"]
            .schemas["HOP"]
            .cortex_search_services["RAG_SEARCH_SERVICE"]
        )
        resp = cortex_search_service.search(
            query=query,
            columns=["SEARCH_COL"],
            limit=self._limit_to_retrieve,
        )
        return [row["SEARCH_COL"] for row in resp.results] if resp.results else []

# Initialize the retriever
retriever = CortexSearchRetriever(session=session, limit_to_retrieve=3)
test_query = "What are some components of the Snowflake Cortex offering? How do they work?"
retrieved_context = retriever.retrieve(query=test_query)
print(retrieved_context)

In [ ]:
import os
os.environ["TRULENS_OTEL_TRACING"] = "1"

In [ ]:
# Create the RAGWithObservability class to structure the RAG pipeline
from snowflake.cortex import complete
from trulens.core.otel.instrument import instrument
from trulens.otel.semconv.trace import SpanAttributes


class RAGWithObservability():
    def __init__(self, llm_model, retriever):
        self.llm_model = llm_model
        self.retriever = retriever
        
#Here we're using the @instrument decorator to trace various stages of our RAG applicaiton
    @instrument (
        span_type=SpanAttributes.SpanType.RETRIEVAL, 
        attributes={
            SpanAttributes.RETRIEVAL.QUERY_TEXT: "query",
            SpanAttributes.RETRIEVAL.RETRIEVED_CONTEXTS: "return",
        })  
    def retrieve_context(self, query: str) -> List[str]:
        return self.retriever.retrieve(query)

    @instrument()
    def augment_prompt(self, query: str, contexts: list) -> str:
     
        prompt = f"""
        You are an expert assistant extracting information from context provided.
        Answer the question based on the context. Be concise and do not hallucinate.
        If you don't have the information, just say so.
        Context: {' '.join(contexts)}
        Question: {query}
        Answer:
        """
        return prompt


    @instrument (span_type=SpanAttributes.SpanType.GENERATION)    
    def generate_completion(self, query: str):
        
        df_response = complete(self.llm_model, query)
        return df_response


    @instrument (
        span_type=SpanAttributes.SpanType.RECORD_ROOT, 
        attributes={
            SpanAttributes.RECORD_ROOT.INPUT: "query",
            SpanAttributes.RECORD_ROOT.OUTPUT: "return",
        })
    def query_app(self, query: str) -> str:
        contexts = self.retrieve_context(query)
        prompt = self.augment_prompt(query, contexts)
        final_response = self.generate_completion(prompt)
        return final_response

In [ ]:
import streamlit as st

#Define LLM classes
llama_rag = RAGWithObservability('llama3.1-8b', retriever)
mistral7b_rag = RAGWithObservability('mistral-7b', retriever)
claude_rag = RAGWithObservability('claude-3-5-sonnet', retriever)

#print Query
print(f"Query: {test_query}")

#Get and print responses
llama_response = llama_rag.query_app(test_query)
st.write(f"**Llama response** -  {llama_response} \n")

mistral_response = mistral7b_rag.query_app(test_query)
st.write(f"**Mistral-7b response** - {mistral_response} \n")

claude_response = claude_rag.query_app(test_query)
st.write(f"**Claude response** -  {claude_response} \n")

## Step 6: Observe and Evaluate LLM Performance with AI Observability (powered by TruLens)

**Adding Observability and Evaluataion to our RAG application**

Here, we enhance the Retrieval-Augmented Generation (RAG) process by introducing observability. Observability ensures that LLM responses can be measured and evaluated based on various feedback metrics, providing insights into the model's performance and areas for improvement.

**How This Works**

We will use a feature called AI Observability to register our recently created applications in Snowflake. This will allow users to pass in prompts to these applications, and trace each step the application takes to Retrieve appropriate context, Augment a system prompt with additional context and Generate a complete answer for the given prompt. 

From there we will use LLM-as-a-Judge based evaluations to measure LLM performance based on **feedback metrics** including:
- **Answer Relevance** - Evaluates how directly the LLM's response addresses the user's prompt.
- **Context Relevance** - Assesses the relevance of the retrieved context to the user's prompt.
- **Groundedness**  - Measures how well the LLM's response is anchored in the retrieved context.
- **Coherance** - Evaluates how logically structured and easy to follow the LLM's response is.

In [ ]:
# Define image in a stage and read the file
image=session.file.get_stream("@SKO_SKORAGHOP_LIVE_PROD.HOP.RAG/AIObservability.jpg", decompress=False).read() 

# Display the image
st.image(image, width=800)

In [ ]:
# from trulens.core import TruSession
from trulens.apps.app import TruApp
from trulens.connectors.snowflake import SnowflakeConnector

tru_snowflake_connector = SnowflakeConnector(snowpark_session=session)

app_name = "test_sko_app_update"
version_num = 'v0'

tru_rag_mistral = TruApp(
    mistral7b_rag,
    app_name=app_name,
    app_version=f"mistral_test_{version_num}",
    connector=tru_snowflake_connector
)

tru_rag_llama = TruApp(
    llama_rag,
    app_name=app_name,
    app_version=f"llama_test_{version_num}",
    connector=tru_snowflake_connector
)

tru_rag_claude = TruApp(
    claude_rag,
    app_name=app_name,
    app_version=f"claude_test_{version_num}",
    connector=tru_snowflake_connector
)

In [ ]:
import pandas as pd

prompts = [
    "What are some metrics to measure the quality of a retrieval system?",
    "Can I have a back-and-forth conversation with Cortex?",
    "Does Snowflake support text-to-sql? What services would support this?",
    "What year was the war of 1812?",
    "Tell me a story about Snowflake Cortex"
]


batch_data = pd.DataFrame({'QUERY': prompts})
batch_data

In [ ]:
from trulens.core.run import Run
from trulens.core.run import RunConfig

mistral_run_config = RunConfig(
    run_name=f"mistral_exp_{version_num}",
    description="questions about snowflake AI cababilities",
    dataset_name="SNOW_RAG_DF1",
    source_type="DATAFRAME",
    label="MISTRAL",
    llm_judge_name = "llama3.1-70b",
    dataset_spec={
        "RECORD_ROOT.INPUT": "QUERY",
    },
)



llama_run_config = RunConfig(
    run_name=f"llama_exp_{version_num}",
    description="questions about snowflake AI cababilities",
    dataset_name="SNOW_RAG_DF1",
    source_type="DATAFRAME",
    label="LLAMA",
    dataset_spec={
        "RECORD_ROOT.INPUT": "QUERY",
    },
    
)


claude_run_config = RunConfig(
    run_name=f"claude_exp_{version_num}",
    description="questions about snowflake AI cababilities",
    dataset_name="SNOW_RAG_DF1",
    source_type="DATAFRAME",
    label="CLAUDE",
    dataset_spec={
        "RECORD_ROOT.INPUT": "QUERY",
    },
    
)

In [ ]:
mistral_run = tru_rag_mistral.add_run(run_config=mistral_run_config)

llama_run = tru_rag_llama.add_run(run_config=llama_run_config)

claude_run = tru_rag_claude.add_run(run_config=claude_run_config)

In [ ]:
mistral_run.start(input_df=batch_data)
print("Finished mistral run")

In [ ]:
llama_run.start(input_df=batch_data)
print("Finished Llama run")

In [ ]:
claude_run.start(input_df=batch_data)
print("Finished Claude run")

In [ ]:
print(f"Mistral: {mistral_run.get_status()}")
print(f"Llama: {llama_run.get_status()}")
print(f"Claude: {claude_run.get_status()}")

In [ ]:
#The following code kicks off LLM-as-a-Judge evals for several metrics

mistral_run.compute_metrics([
    "coherence",
    "answer_relevance",
    "context_relevance",
    "groundedness",
])

In [ ]:
#The following code kicks off LLM-as-a-Judge evals for several metrics

llama_run.compute_metrics([
    "coherence",
    "answer_relevance",
    "context_relevance",
    "groundedness",
])

In [ ]:
#The following code kicks off LLM-as-a-Judge evals for several metrics

claude_run.compute_metrics([
    "coherence",
    "answer_relevance",
    "context_relevance",
    "groundedness",
])

In [ ]:
print(f"Mistral: {mistral_run.get_status()}")
print(f"Llama: {llama_run.get_status()}")
print(f"Claude: {claude_run.get_status()}")

In [ ]:
import streamlit as st

org_name = session.sql('SELECT CURRENT_ORGANIZATION_NAME()').collect()[0][0]
account_name = session.sql('SELECT CURRENT_ACCOUNT_NAME()').collect()[0][0]
db_name = session.sql('SELECT CURRENT_DATABASE()').collect()[0][0]
schema_name = session.sql('SELECT CURRENT_SCHEMA()').collect()[0][0]

st.write(f'https://app.snowflake.com/{org_name}/{account_name}/#/ai-evaluations/databases/{db_name}/schemas/{schema_name}/applications/{app_name.upper()}')

In [ ]:
# Define image in a stage and read the file
image=session.file.get_stream("@SKO_SKORAGHOP_LIVE_PROD.HOP.RAG/AIObsApp.jpg", decompress=False).read() 

# Display the image
st.image(image, width=800)

In [ ]:
image1=session.file.get_stream("@SKO_SKORAGHOP_LIVE_PROD.HOP.RAG/Anthropic.jpg", decompress=False).read() 
st.image(image1, width=800)
image2=session.file.get_stream("@SKO_SKORAGHOP_LIVE_PROD.HOP.RAG/Summary2.jpg", decompress=False).read() 
st.image(image2, width=800)